In [ ]:
!pip install langchain openai GitPython -q

In [8]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_KEY')

In [4]:
from langchain.document_loaders import GitLoader

def file_filter(file_path):
  return file_path.endswith(".mdx")

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain", # save to
    branch="master",
    file_filter=file_filter
)

raw_docs = loader.load()
print(len(raw_docs))

279


In [5]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(raw_docs)

len(docs)

925

In [6]:
!pip install tiktoken -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.7 MB/s eta 0:00:00


In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings

# vectorize
embeddings = OpenAIEmbeddings()

query = "S3からデータを読み込むためのDocumentLoaderはありますか？"

vector = embeddings.embed_query(query)

print(len(vector))
print(vector)

1536
[-0.011905040852273058, 0.003959383902103599, 0.012967150941911237, -0.008671656440569752, -0.00861787814864523, 0.04038706137634194, -0.008974155607354812, -0.021215307058865755, 0.00024031915179674256, 0.0020855667991249594, 0.018123088800819132, 0.015165314409938624, 0.012194096377399582, -0.009518655225155033, -0.014721649089496884, -0.01806931050889461, 0.017047534138199823, -0.0001470484308223216, 0.011421041812887925, -0.039311506713722626, 0.012671373130294148, -0.0067222138474029025, -0.008564100788043302, -0.015434203075593452, -0.016765201830886457, 0.007999434310771386, 0.015393870287972655, -0.03748306714003114, 0.01722231172430933, -0.0009385891231119611, 0.04732438652879774, -0.013746927792913458, 0.004265244677795238, -0.02477808164596157, 0.0010948806542030634, -0.024885636367165426, -0.004349272327604709, -0.01216720723143732, -0.007407879618859804, -0.019951532099801, 0.011790762913256043, -0.013518371914879429, 0.009565710299266394, -0.025342746260588297, -0.01

In [ ]:
!pip install chromadb -q

In [13]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(docs, embeddings)

In [15]:
retriever = db.as_retriever()

context_docs = retriever.get_relevant_documents(query)
print(f"len = {len(context_docs)}")

first_doc = context_docs[0]
print(f"metadata = {first_doc.metadata}")
print(first_doc.page_content)

len = 4
metadata = {'file_name': 'aws.mdx', 'file_path': 'docs/docs/integrations/platforms/aws.mdx', 'file_type': '.mdx', 'source': 'docs/docs/integrations/platforms/aws.mdx'}
See a [usage example](/docs/integrations/text_embedding/sagemaker-endpoint).
```python
from langchain_community.embeddings import SagemakerEndpointEmbeddings
from langchain_community.llms.sagemaker_endpoint import ContentHandlerBase
```

## Document loaders

### AWS S3 Directory and File

>[Amazon Simple Storage Service (Amazon S3)](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-folders.html)
> is an object storage service.
>[AWS S3 Directory](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-folders.html)
>[AWS S3 Buckets](https://docs.aws.amazon.com/AmazonS3/latest/userguide/UsingBucket.html)

See a [usage example for S3DirectoryLoader](/docs/integrations/document_loaders/aws_s3_directory).

See a [usage example for S3FileLoader](/docs/integrations/document_loaders/aws_s3_file).

```pytho

In [17]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm=chat, chain_type="stuff", retriever=retriever)

result = qa_chain.run(query)

print(result)

はい、AWS S3からデータを読み込むためのDocument Loaderがあります。具体的には、`S3DirectoryLoader`と`S3FileLoader`が使用できます。これらのローダーを使用すると、AWS S3からデータを読み込んで`Document`として取得することができます。
